In [1]:
import pandas as pd

colnames = ['speaker', 'discipline', 'description' , 'why']
df = pd.read_csv('../data/speakers.csv', names=colnames)

In [2]:
df.head()

,speaker,discipline,description,why
0,Alwar Balasubramaniam,Artist,"Sculptor, painter and printmaker Alwar Balasub...",Why you should listen\nAlwar Balasubramaniam's...
1,AJ Jacobs,Author,Immersing himself in alternate lifestyles and ...,Why you should listen\nAJ Jacobs' writings sta...
2,Aakash Odedra,Choreographer,Aakash Odedra sets raw ancient dance forms fro...,Why you should listen\nBased in the UK with a ...
3,Aaron Huey,Photographer,"Photographer, adventurer and storyteller Aaron...",Why you should listen\nAaron Huey is a masthea...
4,Aaron Koblin,Data artist,Aaron Koblin is an artist specializing in data...,Why you should listen\nAaron Koblin finds art ...


In [ ]:

female = ["she", "her", "hers"]
male = ["he", "him", "his"]

# pseudocode

if one of the items in female is in the column: 
    put an F in the gender column